# Testowanie, czy dane mają rozkład normalny

Poniższy notatnik jest inspirowany wpisami [Jasona Brownlee](https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/) oraz [Łukasza Prokulskiego](https://blog.prokulski.science/index.php/2019/01/24/rozklad-normalny-hipotezy-i-regresja-logistyczna/) dotyczącymi testowania rozkładu danych.

Do analiz wykorzystam zbiór danych dotyczących wzrostu i wagi amerykańskich nastolatków [link](https://www.statcrunch.com/app/index.php?dataid=1757964).

In [1]:
import pandas as pd

In [2]:
# Read data (already downloaded)
df = pd.read_csv("../data/NLSY97.csv", delimiter=" ")
df.head(3)

,Gender,Age,Height (inches),Weight (lbs),How would you describe your weight?
0,Female,18,67,150,Slightly overweight
1,Male,17,67,140,About the right weight
2,Female,16,67,100,Slightly underweight


In [3]:
# Convert measurement units
df["Height (cm)"] = df["Height (inches)"] * 2.54     # 1 inch = 2.54cm
df["Weight (kg)"] = df["Weight (lbs)"] * 0.45359237  # 1 pound ~= 0.45kg

In [4]:
# Take only what we are interested in
df = df[["Gender", "Height (cm)", "Weight (kg)"]]
df.head(3)

,Gender,Height (cm),Weight (kg)
0,Female,170.18,68.038855
1,Male,170.18,63.502932
2,Female,170.18,45.359237


TODO: 
* dodać wyjaśnienie, dlaczego testowanie rozkładu jest ważne.